<a href="https://colab.research.google.com/github/ShanChartrand/NECDataCleaning/blob/main/NEC_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing file

In [143]:
!pip install pandas
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing libraries

In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import requests

Viewing original number of rows and columns

In [145]:
# Count the number of rows and columns
num_rows = df.shape[0]
num_columns = df.shape[1]

# Print the results
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

Number of rows: 19928
Number of columns: 31


Removing rows 2-8064, before metrics such as Jitter, Latency, and Packet loss were reliably measured

In [146]:
# Read the CSV file
df = pd.read_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv')

# Remove rows 2 through 8064
df.drop(df.index[1:8064], inplace=True)

# Save the updated DataFrame 
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)

Counting the new number of rows & columns after removing rows 2-8064

In [147]:
# Count the number of rows and columns
num_rows = df.shape[0]
num_columns = df.shape[1]

# Print the results
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

Number of rows: 11865
Number of columns: 31


Looking at the % of missing items in columns

In [148]:
# Calculate the percentage of missing cells in each column
missing_percentages = df.isna().sum() / len(df) * 100

# Print the result
print(missing_percentages)

TEST ID                       0.000000
TEST DATE / UTC HOUR          0.000000
GEO LATITUDE                  0.000000
GEO LONGITUDE                 0.000000
DOWNLOAD SPEED                0.000000
UPLOAD SPEED                  0.000000
JITTER                        0.008428
LATENCY                       0.008428
PACKET LOSS                   0.008428
MOBILE TEST                   0.000000
ISP                           0.000000
CLIENT IP                     0.000000
CLIENT CITY                   0.000000
CLIENT PROVINCE               0.000000
CLIENT POSTAL CODE            0.000000
AS NUMBER                     0.008428
IPV6 SUPPORTED                0.000000
DNSSEC SUPPORTED              0.000000
TEST SERVER                   0.000000
browser_test_url              0.008428
CONTRACTED DOWNLOAD SPEED    75.347661
CONTRACTED UPLOAD SPEED      80.311842
CONTRACTED DATA CAP          87.265065
CONTRACTED MONTHLY COST      79.241466
SERVICE TYPE                 73.881163
USER SATISFACTION        

Removing columns with greater than 68% missing values, and then listing the remaining number of rows & columns

In [149]:
# List of columns to remove
columns_to_remove = [
    'CONTRACTED DOWNLOAD SPEED',
    'CONTRACTED UPLOAD SPEED',
    'CONTRACTED DATA CAP',
    'CONTRACTED MONTHLY COST',
    'SERVICE TYPE',
    'USER SATISFACTION',
    'CONNECTION TYPE',
    'USER COMMENTS',
    'HEATMAP_PROJECT_AREA'
]

# Remove the columns
df.drop(columns=columns_to_remove, inplace=True)

# Save the updated DataFrame 
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)

# Count the number of rows and columns
num_rows = df.shape[0]
num_columns = df.shape[1]

# Print the results
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

Number of rows: 11865
Number of columns: 22


Checking for duplicates in the TEST ID column

In [150]:
# Count the number of duplicated TEST ID values
num_duplicates = df.duplicated(subset='TEST ID', keep=False).sum()

# Print the number of duplicates
print(f"There are {num_duplicates} rows with duplicated TEST ID values.")

There are 0 rows with duplicated TEST ID values.


Looking for duplicate rows

In [151]:
# Find duplicate rows based on all columns
duplicates = df[df.duplicated()]

# Display the duplicate rows (if any)
if len(duplicates) > 0:
    print(f"Found {len(duplicates)} duplicate rows:")
    print(duplicates)
else:
    print("No duplicate rows found.")

No duplicate rows found.


Looking at the % of missing items in columns

In [152]:
# Calculate the percentage of missing cells in each column
missing_percentages = df.isna().sum() / len(df) * 100

# Print the result
print(missing_percentages)

TEST ID                  0.000000
TEST DATE / UTC HOUR     0.000000
GEO LATITUDE             0.000000
GEO LONGITUDE            0.000000
DOWNLOAD SPEED           0.000000
UPLOAD SPEED             0.000000
JITTER                   0.008428
LATENCY                  0.008428
PACKET LOSS              0.008428
MOBILE TEST              0.000000
ISP                      0.000000
CLIENT IP                0.000000
CLIENT CITY              0.000000
CLIENT PROVINCE          0.000000
CLIENT POSTAL CODE       0.000000
AS NUMBER                0.008428
IPV6 SUPPORTED           0.000000
DNSSEC SUPPORTED         0.000000
TEST SERVER              0.000000
browser_test_url         0.008428
HEATMAP_MUNICIPALITY     0.000000
HEATMAP_CENSUS_TRACTS    0.000000
dtype: float64


As per above, we can see that the columns that have missing items each have exactly 0.008428%, which indicates there was a test interruption or failure with CIRA. Will remove those rows then confirm the % of missing items per column.

In [153]:
# List of columns to check for missing values
columns_to_check = ['JITTER', 'LATENCY', 'PACKET LOSS', 'AS NUMBER', 'browser_test_url']

# Remove rows with missing values in specified columns
df.dropna(subset=columns_to_check, inplace=True)

# Save the updated DataFrame 
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)

# Calculate the percentage of missing cells in each column
missing_percentages = df.isna().sum() / len(df) * 100

# Print the result
print(missing_percentages)

TEST ID                  0.0
TEST DATE / UTC HOUR     0.0
GEO LATITUDE             0.0
GEO LONGITUDE            0.0
DOWNLOAD SPEED           0.0
UPLOAD SPEED             0.0
JITTER                   0.0
LATENCY                  0.0
PACKET LOSS              0.0
MOBILE TEST              0.0
ISP                      0.0
CLIENT IP                0.0
CLIENT CITY              0.0
CLIENT PROVINCE          0.0
CLIENT POSTAL CODE       0.0
AS NUMBER                0.0
IPV6 SUPPORTED           0.0
DNSSEC SUPPORTED         0.0
TEST SERVER              0.0
browser_test_url         0.0
HEATMAP_MUNICIPALITY     0.0
HEATMAP_CENSUS_TRACTS    0.0
dtype: float64


We had 11865 rows, let's see how many were removed

In [154]:
# Count the number of rows
num_rows = df.shape[0]

# Print the results
print("Number of rows:", num_rows)

Number of rows: 11863


We can see that only 2 rows were removed. Now to determine the # of unique ISPs

In [155]:
unique_isp_count = len(df['ISP'].unique())
print("Number of unique ISPs:", unique_isp_count)

Number of unique ISPs: 84


Now we must standardize the ISPs and then count the # of unique ISPs after standardizatioin



In [156]:
def standardize_isp(isp):
    replacements = {
        "Access Communications  Co-operative Limited": "Access Communications Co-operative Limited",
        "Amanah Tech": "Amanah Tech Inc.",
        "Amazon.com, Inc.": "Amazon Technologies Inc.",
        "Bell MTS": "Bell Canada",
        "Bell Mobility": "Bell Canada",
        "Bell Mobility, Inc.": "Bell Canada",
        "Bell MTS Data Centres G.P": "Bell Canada",
        "CIK Telecom": "CIK Telecom INC",
        "Commstream Communications": "Commstream Communications Inc",
        "Corridor Communications, INC.": "Corridor Communications",
        "Manitoba First Nations Education Resource Centre Inc.": "Manitoba First Nations Education Resource Centre",
        "Manitoba Hydro International LTD": "Manitoba Hydro",
        "Manitoba Hydro Telecom": "Manitoba Hydro",
        "MTS Allstream": "MTS Inc.",
        "MTS Inc": "MTS Inc.",
        "MTS Allstream Inc.": "MTS Inc.",
        "MTS Data Centres": "MTS Inc.",
        "Red River College of Applied Arts": "Red River College",
        "Red River College of Applied Arts, Science and Tec": "Red River College",
        "Rogers Cable": "Rogers Communications",
        "Rogers Communications Canada Inc.": "Rogers Communications",
        "Full Throttle Networks": "Full Throttle Networks Inc.",
        "Postmedia Network": "Postmedia Network Inc",
        "Radiant Communications Canada Ltd.": "Radiant Communications Canada",
        "Shaw Communications Inc.": "Shaw Communications",
        "Shaw Telecom G.P.": "Shaw Communications",
        "SpaceX Services": "SpaceX",
        "SpaceX Starlink": "SpaceX",
        "TekSavvy Solutions, Inc.": "TekSavvy Solutions",
        "TELUS Communications Inc": "Telus Communications",
        "TELUS Communications Inc.": "Telus Communications",
        "TeraGo Networks Inc": "TeraGo Networks",
        "TeraGo Networks Inc.": "TeraGo Networks",
        "Total Server Solutions L.L.C": "Total Server Solutions L.L.C",
        "Voyageur Internet Inc": "Voyageur Internet",
        "Xplornet Communications Inc": "Xplornet Communications",
        "Xplornet Communications Inc.": "Xplornet Communications"
    }
    
    return replacements.get(isp, isp)

# Example usage:
# Assuming you have a DataFrame named 'df' with a column named 'ISP'
df['ISP'] = df['ISP'].apply(standardize_isp)

unique_isp_count = len(df['ISP'].unique())
print("Number of unique ISPs:", unique_isp_count)

Number of unique ISPs: 73


I'm going to list all the different ISPs

In [157]:
unique_isps = df['ISP'].unique()
for isp in unique_isps:
    print(isp)

Shaw Communications
Bell Canada
WiBand Communications
TekSavvy Solutions
Altima Telecom
Cipherkey Exchange Corp.
Red River College
CIK Telecom INC
Voyageur Internet
Xplornet Communications
MERLIN
The Exchange Global Server Center
Amanah Tech Inc.
Primus Telecommunications Canada Inc.
ViaNetTV Inc
Rogers Communications
City of Winnipeg
Stafford Communications, Inc.
Full Throttle Networks Inc.
MTS Inc.
Total Server Solutions L.L.C.
Questzone.Net, Inc.
Telus Communications
RBC Convention Centre Winnipeg
Cogent Communications
BIGPIPE
University of Manitoba
Corus Entertainment Inc
Postmedia Network Inc
Fastnet Communications
T. Grand Networks Inc.
Netminders Data Solution
Manitoba Hydro
TeraGo Networks
Alberta Health Services
South Winnipeg Technical Centre
Swift High Speed.com
VPN
GloboTech Communications
Corridor Communications
LES.NET
VOI NET INC.
Packet Exchange Limited
Performive LLC
Radiant Communications Canada
SpaceX
Access Communications Co-operative Limited
BH Telecom Corp.
MRNet


Looking at for upload/download speeds of 0 mbps

In [158]:
# Count the number of cells equal to zero in the 'DOWNLOAD SPEED' column
download_speed_zeros = (df['DOWNLOAD SPEED'] == 0).sum()

# Count the number of cells equal to zero in the 'UPLOAD SPEED' column
upload_speed_zeros = (df['UPLOAD SPEED'] == 0).sum()

# Print the counts
print("Number of cells equal to zero in 'DOWNLOAD SPEED' column:", download_speed_zeros)
print("Number of cells equal to zero in 'UPLOAD SPEED' column:", upload_speed_zeros)

Number of cells equal to zero in 'DOWNLOAD SPEED' column: 0
Number of cells equal to zero in 'UPLOAD SPEED' column: 5


I now need to remove the 5 rows that have upload speeds of 0, and then list how many cells = 0 to confirm removal was successful

In [159]:
# Count the number of cells equal to zero in the 'UPLOAD SPEED' column before removal
upload_speed_zeros_before = (df['UPLOAD SPEED'] == 0).sum()

# Remove rows where UPLOAD SPEED is equal to 0
df = df[df['UPLOAD SPEED'] != 0]

# Count the number of cells equal to zero in the 'UPLOAD SPEED' column after removal
upload_speed_zeros_after = (df['UPLOAD SPEED'] == 0).sum()

# Save the updated DataFrame 
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)

# Print the count before and after removal
print("Number of cells equal to zero in 'UPLOAD SPEED' column before removal:", upload_speed_zeros_before)
print("Number of cells equal to zero in 'UPLOAD SPEED' column after removal:", upload_speed_zeros_after)

Number of cells equal to zero in 'UPLOAD SPEED' column before removal: 5
Number of cells equal to zero in 'UPLOAD SPEED' column after removal: 0


**THIS IS TO BE USED AFTER ALL OUTLIERS ARE REMOVED**

Deciding if mean or median should be used as the measure of central tendency for UPLOAD SPEED. Looking at the variability/skewness of the data

In [161]:
# Extract the UPLOAD SPEED column as a Series
upload_speed = df['UPLOAD SPEED']

# Calculate the mean and median
mean = upload_speed.mean()
median = upload_speed.median()

# Calculate other statistical measures
range_val = upload_speed.max() - upload_speed.min()
std_dev = upload_speed.std()
skewness = upload_speed.skew()


# Find the minimum and maximum values
min_upload_speed = upload_speed.min()
max_upload_speed = upload_speed.max()

# Print the results
print("Min:", min_upload_speed)
print("Max:", max_upload_speed)
print("Mean:", mean)
print("Median:", median)
print("Range:", range_val)
print("Standard Deviation:", std_dev)
print("Skewness:", skewness)

Min: 0.01
Max: 932.9
Mean: 21.611705177938944
Median: 7.37
Range: 932.89
Standard Deviation: 52.785077537631146
Skewness: 7.410393613016622


Determining central tendency for DOWNLOAD SPEED

In [162]:
# Extract the DOWNLOAD SPEED column as a Series
download_speed = df['DOWNLOAD SPEED']

# Calculate the mean and median
mean = download_speed.mean()
median = download_speed.median()

# Calculate other statistical measures
range_val = download_speed.max() - download_speed.min()
std_dev = download_speed.std()
skewness = download_speed.skew()

# Find the minimum and maximum values
min_download_speed = download_speed.min()
max_download_speed = download_speed.max()

# Print the results
print("Min:", min_download_speed)
print("Max:", max_download_speed)
print("Mean:", mean)
print("Median:", median)
print("Range:", range_val)
print("Standard Deviation:", std_dev)
print("Skewness:", skewness)

Min: 0.01
Max: 941.74
Mean: 95.04424439197166
Median: 53.545
Range: 941.73
Standard Deviation: 113.19664084838082
Skewness: 2.1693861760629525


Central tendency for JITTER

In [163]:

# Extract the JITTER column as a Series
jitter = df['JITTER']

# Calculate the mean and median
mean = jitter.mean()
median = jitter.median()

# Calculate other statistical measures
range_val = jitter.max() - jitter.min()
std_dev = jitter.std()
skewness = jitter.skew()

# # Find the minimum and maximum values
min_jitter = jitter.min()
max_jitter = jitter.max()

# Print the results
print("Min:", min_jitter)
print("Max:", max_jitter)
print("Mean:", mean)
print("Median:", median)
print("Range:", range_val)
print("Standard Deviation:", std_dev)
print("Skewness:", skewness)

Min: 0.01
Max: 4368.64
Mean: 7.071970821386405
Median: 2.02
Range: 4368.63
Standard Deviation: 50.98203510406158
Skewness: 59.35629910273703


Examing variabilty in LATENCY

In [164]:
# Extract the LATENCY column as a Series
latency = df['LATENCY']

# Calculate the mean and median
mean = latency.mean()
median = latency.median()

# Calculate other statistical measures
range_val = latency.max() - latency.min()
std_dev = latency.std()
skewness = latency.skew()

# # Find the minimum and maximum values
min_latency = latency.min()
max_latency = latency.max()

# Print the results
print("Min:", min_latency)
print("Max:", max_latency)
print("Mean:", mean)
print("Median:", median)
print("Range:", range_val)
print("Standard Deviation:", std_dev)
print("Skewness:", skewness)

Min: 0.1
Max: 4294967.3
Mean: 410.3985579355708
Median: 40.92
Range: 4294967.2
Standard Deviation: 39441.13730441986
Skewness: 108.89432015143929


Checking PACKET LOSS variabilty

In [165]:
# Extract the PACKET LOSS column as a Series
packet_loss = df['PACKET LOSS']

# Calculate the mean and median
mean = packet_loss.mean()
median = packet_loss.median()

# Calculate other statistical measures
range_val = packet_loss.max() - packet_loss.min()
std_dev = packet_loss.std()
skewness = packet_loss.skew()

# # Find the minimum and maximum values
min_packet_loss = packet_loss.min()
max_packet_loss = packet_loss.max()

# Print the results
print("Min:", min_packet_loss)
print("Max:", max_packet_loss)
print("Mean:", mean)
print("Median:", median)
print("Range:", range_val)
print("Standard Deviation:", std_dev)
print("Skewness:", skewness)

Min: 0.0
Max: 47.85
Mean: 2.0036667228875022
Median: 0.05
Range: 47.85
Standard Deviation: 4.389391448549163
Skewness: 3.4920742969507836
